### Example 1: Complex ReAct Agent for Product Inquiry and Stock Availability with Memory and Dynamic Decision-Making

In [1]:
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver

In [3]:
#Define tools
def product_info(product_name: str) -> str:
    """Fetch product information given a product name."""
    product_catalog = {
        "Laptop": "A high-performance laptop with 16GB RAM and 512GB SSD.",
        "Smartphone": "A latest model smartphone with excellent camera features.",
        "Headphones": "Noise-cancelling over-ear headphones with superior sound quality."
    }
    return product_catalog.get(product_name, "Product not found.")

def check_stock(product_name: str) -> str:
    """Check stock availability for a given product."""
    stock_data = {
        "Laptop": "In Stock",
        "Smartphone": "Out of Stock",
        "Headphones": "In Stock"
    }
    return stock_data.get(product_name, "Product not found.")


#Initialize the memory saver for single session memory
checkpointer = MemorySaver()

#Initialize the language model
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

#Create the ReAct agent with the defined tools and memory
tools = [product_info, check_stock]
graph = create_react_agent(model=llm, tools=tools, checkpointer=checkpointer)

#Setup thread configuration for single session memory
config = {"configurable": {"thread_id": "thread-3"}}

#User input : initiale inquiry about product information 
inputs = {"messages": [("user", "Tell me about the Laptop. ")]}

messages = graph.invoke(inputs,config=config)
for message in messages["messages"]:
    print(message.content)

/var/folders/cr/q3l9dqfn2hb1v4cdl1m__hc80000gn/T/ipykernel_68754/858744498.py:29: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  graph = create_react_agent(model=llm, tools=tools, checkpointer=checkpointer)


Tell me about the Laptop. 

A high-performance laptop with 16GB RAM and 512GB SSD.
The laptop is a high-performance device equipped with 16GB of RAM and a 512GB SSD, making it suitable for various tasks, including gaming, content creation, and multitasking. If you need more specific information or details about a particular model, feel free to ask!


In [4]:
inputs2 = {"messages": [("user", "Is it available in stock? ")]}

messages = graph.invoke(inputs2,config=config)
for message in messages["messages"]:
    print(message.content)

Tell me about the Laptop. 

A high-performance laptop with 16GB RAM and 512GB SSD.
The laptop is a high-performance device equipped with 16GB of RAM and a 512GB SSD, making it suitable for various tasks, including gaming, content creation, and multitasking. If you need more specific information or details about a particular model, feel free to ask!
Is it available in stock? 

In Stock
Yes, the laptop is currently in stock. If you have any other questions or need assistance with purchasing, let me know!


## Example 2: ReAct Agent with Multi-Step Reasoning and Dynamic Action Sub-Graaphs

## Plan:
### 1-Parent graph:Responsible for high-level reasoning and orchestrating multiple sub-graphs
### 2-Reasoning phase:Dynamically decide which sub-graphs to call based on user input
### 3-Sub-Graphs:Handle specific actions including tool calls or acting on the user's query
### 4-Memory:Store and recaall data in the sub-graphs 

In [5]:
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, MessagesState, START, END
from typing import TypedDict

In [11]:
#Define shared state for the agent and sub-graphs
class ReActAgentState(TypedDict):
    messages: str
    action: str
    sub_action: str


#Reasoning node 1: Determine which action the agent should take based on the query 
def reasoning_node(state: ReActAgentState):
    query = state["message"]
    if "weather" in query:
        return {"action": "fetch_weather"}
    elif "news" in query:
        return {"action": "fetch_news"}
    elif "recommend" in query:
        return {"action": "recommendation", "sub_action": "book"}
    else:
        return {"action": "unknown"}
    

#Sub-graph for fetching news information (action phase)
def news_subgraph_node(state: ReActAgentState):
    return {"message": "Here are the latest news headlines..."}

#Sub-graph for fetching weather information (action phase)
def weather_subgraph_node(state: ReActAgentState):
    return {"message": "The current weather is sunny with a temperature of 75°F."}


#Sub-graph for providing recommendations (action phase)
def recommendation_subgraph_node(state: ReActAgentState):
    if state.get("sub_action") == "book":
        return {"message": "I recommend 'The Great Gatsby' by F. Scott Fitzgerald."}
    else:
        return {"message": "I recommend trying out the new Italian restaurant downtown."}
    

#Build sub-graph for fetching weather information
weather_subgraph_builder = StateGraph(ReActAgentState)
weather_subgraph_builder.add_node("weather_action", weather_subgraph_node)
weather_subgraph_builder.set_entry_point("weather_action")
weather_subgraph = weather_subgraph_builder.compile()

#Build sub-graph for fetching news information
news_subgraph_builder = StateGraph(ReActAgentState)
news_subgraph_builder.add_node("news_action", news_subgraph_node)
news_subgraph_builder.set_entry_point("news_action")
news_subgraph = news_subgraph_builder.compile()

#Build sub-graph for providing recommendations
recommendation_subgraph_builder = StateGraph(ReActAgentState)
recommendation_subgraph_builder.add_node("recommendation_action", recommendation_subgraph_node)
recommendation_subgraph_builder.set_entry_point("recommendation_action")
recommendation_subgraph = recommendation_subgraph_builder.compile()


#Define dynamic reasoning node to select and invoke sub-graphs based on the action
def reasoning_state_manager(state: ReActAgentState):
    if state["action"] == "fetch_weather":
        return weather_subgraph
    elif state["action"] == "fetch_news":
        return news_subgraph
    elif state["action"] == "recommendation":
        return recommendation_subgraph
    else:
        return None
    
#Create the parent ReAct agent graph
parent_builder = StateGraph(ReActAgentState)
parent_builder.add_node("reasoning", reasoning_node)
parent_builder.add_node("action_dispathch", reasoning_state_manager)

#Define edges in the parent graph
parent_builder.add_edge(START, "reasoning")
parent_builder.add_edge("reasoning", "action_dispathch")

#Compile the parent graph
react_agent_garph = parent_builder.compile()



In [ ]:
#Test the agent with the wweather query
inputs_weather = {"message": "What's the weather like today?"}
result_weather = react_agent_garph.invoke(inputs_weather)
print(result_weather["message"])


#Test the agent with the news query
inputs_news = {"message": "Give me the latest news headlines."}
result_news = react_agent_garph.invoke(inputs_news)
print(result_news["message"])